In [90]:
import nibabel as nib
import numpy as np

trns = nib.load("../shared/NSD/nsddata/ppdata/subj01/transforms/MNI-to-func1pt8.nii.gz")
print(trns.shape)
print(trns.affine)

mask = nib.load("../shared/NSD/marstpjparcellation_prob_TPJp.nii.gz")
print(mask.shape)

(81, 104, 83, 3)
[[  1.79999995   0.           0.         -72.        ]
 [  0.           1.79999995   0.         -92.69999695]
 [  0.           0.           1.79999995 -73.79999542]
 [  0.           0.           0.           1.        ]]
(182, 218, 182)


In [91]:
arr = trns.get_fdata()

In [110]:
row_ii = arr[:,:,:,0].flatten()
round_row_ii = row_ii.round().astype(int)
#round_row_ii=round_row_ii-1

col_ii = arr[:,:,:,1].flatten()
round_col_ii = col_ii.round().astype(int)
#round_col_ii=round_col_ii-1

sli_ii = arr[:,:,:,2].flatten()
round_sli_ii = sli_ii.round().astype(int)
#round_sli_ii=round_sli_ii-1

In [114]:
condition = (round_row_ii>=182) | (round_row_ii<0)
row_invalid_indices = np.argwhere(condition)
round_row_ii[row_invalid_indices] = 0

condition = (round_col_ii>=218) | (round_col_ii<0)
col_invalid_indices = np.argwhere(condition)
round_col_ii[col_invalid_indices] = 0

condition = (round_sli_ii>=182) | (round_sli_ii<0)
sli_invalid_indices = np.argwhere(condition)
round_sli_ii[sli_invalid_indices] = 0

In [132]:
func_mask = mask_data[(round_row_ii, round_col_ii, round_sli_ii)]
#.reshape((81, 104, 83))
(func_mask.shape, round_row_ii.shape)
func_mask[row_invalid_indices] = 0
func_mask[col_invalid_indices] = 0
func_mask[sli_invalid_indices] = 0

func_mask = func_mask.reshape((81, 104, 83))

In [138]:
import ipyvolume as ipv
import s3fs
ipv.quickvolshow(func_mask)



In [142]:
nii = nib.Nifti1Image(func_mask, check.affine)
nii.to_filename('TPJfunc')

In [139]:
fs = s3fs.S3FileSystem(anon=True)
ll = fs.ls('natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR')

betaslist = [l for l in ll if l.split('/')[6].startswith("betas") and l.endswith("nii.gz")]
fname = betaslist[0] # which session?
fs.get(fname, "tmp.nii.gz")

check = nib.load("tmp.nii.gz")
ipv.quickvolshow(check.get_fdata())

TypeError: _grow_limits() takes 3 positional arguments but 4 were given